## 이미지 썸네일로부터 색상정보 추출
### extcolors 모듈 사용

In [ ]:
%pip install pillow

In [ ]:
%pip install extcolors

In [ ]:
import pandas as pd
import requests
from io import BytesIO
from PIL import Image
import extcolors

In [ ]:
# CSV 파일 로드
csv_path = r"C:\Users\YH\Desktop\데이터분석가부트캠프\practice\HAYEONGHAN705\1st_project\csv\videos_with_captions_full_0312_0322.csv"
df = pd.read_csv(csv_path)

def extract_colors_from_url(url, tolerance=12, limit=5):
    try:
        # 이미지 다운로드
        response = requests.get(url)
        response.raise_for_status()  # 오류 발생 시 예외 발생
        img = Image.open(BytesIO(response.content))
        
        # extcolors로 색상 추출
        # colors: [((R, G, B), 픽셀수), ...], pixel_count: 사용된 총 픽셀수
        colors, pixel_count = extcolors.extract_from_image(img, tolerance=tolerance, limit=limit)
        
        # 리스트에 포함된 색상의 총 픽셀 수 합산
        total = sum(count for (rgb, count) in colors)
        
        # 각 색상의 RGB, 픽셀 수, 그리고 전체 대비 퍼센트 계산
        colors_with_percentage = [(rgb, count, round((count / total * 100), 2)) for (rgb, count) in colors]
        return colors_with_percentage
    except Exception as e:
        print(f"URL 처리 중 오류 발생: {url}\n오류 메시지: {e}")
        return None

# thumbnailURL 열의 각 URL에서 색상 추출 후 새로운 열에 저장
df['extracted_colors'] = df['thumbnailURL'].apply(extract_colors_from_url)

# 결과 CSV 저장
output_csv_path = r"C:\Users\YH\Desktop\데이터분석가부트캠프\practice\HAYEONGHAN705\1st_project\csv\videos_with_color_captions_0312_0322.csv"
df.to_csv(output_csv_path, index=False, encoding='utf-8-sig')

print("모든 이미지의 색상 추출이 완료되었습니다.")